Notes: 

ZHECHO :
    1. function "skewness" applies ^3 is that correct?, additionally currently the skewness is not applied to the data (see code below)
    2. counter added to gradient decent
    3. Why values 0 and 1 are more suitable? The helpers use 1 and -1 as well as the competition. Trying linear regression with 1 and -1 values, doesn't work, diverges.
    4. I think we are currently applying logistic regression, because of sigmoid function
    5. Singular value decomposition doesn't work with the first load
    6. Why do we need encoding?
    7. The best method turns out to be mean imputation on the test (competition) set, maybe the outliers is the reason
    8. Competition insights : Turns out that submitting only "-1" entries gives accuracy of 0.66, which means that 2/3 of the data is "background" and 1/3 "signal". Note that the f1-score for the submission was zero. Imputing the data using the mean value outputs an accuracy of 0.697, which is a slight improvement. F1-score is 0.634. Interestingly, even though the kmeans is more accurate on the validation set, it seems that the on the test data it scores less than 0.697. It seems that the method predicts that there are too many "signals" (at least 60k more than anticipated)
    9. refactorization of the code will be done at the end, in case you create any implementation try to leave small comments to ensure readability of code
    
ANTHONY :
    05/10/20 :
        - is_cat() and build_poly() functions added.
        - Small correction in outliers() function (typo: "x_train" -> "x").
        - Putting less useful codes as comments.
        - The comments below must be taken into account after
          the basic problems have been solved. -> See ZHECHO's comments. 
        - Implementation of the build_poly() functions at the appropriate place (What is your opinion?).
          Because the more we increase the dimensions, the more unstable the gradient descent is
          (because of the change of scale). Should we 1. augment the data, 2. scale, 3. PCA?
          Or maybe 1. PCA, 2. augment, 3. scale? ...
        - We need to create a PCA() function that keeps only the useful sub-linear representation of our data.
          In this way, we will have orthogonal dimensions and dimensionality reduction -> faster convergence.
          
ANTHONY :
    06/10/20 :
        - threshold() function added.
        - Change of strategy : imputation, augmentation, scaling, PCA.

# Libraries, functions and data

## Libraries

In [ ]:
import numpy as np
from proj1_helpers import *
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import sweetviz as sv

## Functions

In [ ]:
def gaussian_scaling(x):
    mean_ = np.mean(x, axis=0)
    x_scaled = x - mean_
    std_ = np.std(x, axis=0)
    x_scaled = x_scaled / std_
    return x_scaled, mean_, std_


def col_na_omit(x, tol=1):
    index = sum(np.isnan(x))/(x.shape[0]) > tol
    x = x[:, ~index]
    return x


def row_na_omit(y, x):
    index = np.isnan(x).any(axis=1)
    x_no_na = x[~index]
    y_no_na = y[~index]
    return y_no_na, x_no_na


def outliers(y, x, clean_data, quantile=3):
    print(f"*****************************************************")
    print(f"                                                     ")
    print(f"WARNING : Outliers must be removed before imputation.")
    print(f"                                                     ")
    print(f"IGNORE : RuntimeWarning: invalid value encountered...",
          "\n", "This is due to NAs in the data set.")
    print(f"                                                     ")
    print(f"*****************************************************")
    m = np.mean(clean_data, axis=0)
    s = np.std(clean_data, axis=0)
    lower = m - quantile * s
    upper = m + quantile * s

    for i in tqdm(range(clean_data.shape[1])):

        index = clean_data[:, i] < lower[i]
        clean_data = np.delete(clean_data, index*1, axis=0)

        index = x[:, i] < lower[i]
        x = np.delete(x, index*1, axis=0)
        y = np.delete(y, index*1, axis=0)

        index = clean_data[:, i] > upper[i]
        clean_data = np.delete(clean_data, index*1, axis=0)

        index = x[:, i] > upper[i]
        x = np.delete(x, index*1, axis=0)
        y = np.delete(y, index*1, axis=0)

    return y, x, clean_data


def mean_imputation(na_data, clean_data):
    null, mean_x, std_x = gaussian_scaling(clean_data)
    na_data = (na_data - mean_x)/std_x
    na_data = np.nan_to_num(na_data, nan=0)
    return(na_data)


def median_imputation(na_data, clean_data):
    null, mean_x, std_x = gaussian_scaling(clean_data)
    na_data = (na_data - mean_x)/std_x
    median = np.median(null, axis=0)
    for i in range(na_data.shape[1]):
        na_data[:, i] = np.nan_to_num(na_data[:, i], nan=median[i])
    return(na_data)


def skewness(x):
    mean_x = np.mean(x, axis=0)
    std_x = np.std(x, axis=0)
    kurt = np.mean(((x - mean_x)/std_x)**3, axis=0)
    return kurt


def min_max_scaling(x):
    min_ = np.min(x, axis=0)
    max_ = np.max(x, axis=0)
    x_scaled = (x - min_)/(max_ - min_)
    return x_scaled, min_, max_


def split_data(x, y, ratio, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # generate random indices
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[: index_split]
    index_te = indices[index_split:]
    # create split
    x_tr = x[index_tr]
    x_te = x[index_te]
    y_tr = y[index_tr]
    y_te = y[index_te]
    return x_tr, x_te, y_tr, y_te


def pairwise(p, q):
    return np.sqrt(np.sum((p[:, np.newaxis, :]-q[np.newaxis, :, :])**2, axis=2))


def random_sample(x, length):
    num_row = x.shape[0]
    indices = np.random.permutation(num_row)
    sample = x[indices][:length]
    return sample


def stochastic_kmeans_imputation(na_data, clean_data, neighbors=10, length=1000):
    clean_data, mean_x, std_x = gaussian_scaling(clean_data)
    na_data = (na_data - mean_x)/std_x
    sample = random_sample(x=clean_data, length=length)
    for i in tqdm(range(na_data.shape[0])):
        condition = np.isnan(na_data[i, :])
        if len(condition) > 0:
            index = np.where(condition)
            candidate = np.delete(na_data[i], index)
            neighborhood = np.delete(sample, index, axis=1)
            distances = pairwise(candidate.reshape(
                (1, len(candidate))), neighborhood)
            nearest_index = np.argsort(distances)[0][:neighbors]
            na_data[i, index] = np.mean(sample[nearest_index], axis=0)[index]
    return(na_data)


def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


def sigmoid(t):
    return 1.0 / (1 + np.exp(-t))


def calculate_loss(y, tx, w):
    pred = sigmoid(tx.dot(w))
    loss = (1/len(y)) * y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))
    return (1/len(y))*np.squeeze(- loss)


def calculate_gradient(y, tx, w):
    y = np.array(y)
    tx = np.array(tx)
    w = np.array(w)
    pred = sigmoid(tx.dot(w))
    grad = tx.T.dot(pred - y)
    return (1/len(y))*grad


def gradient_descent(y, tx, w, max_iter, gamma, eps=1e-4):
    counter = 0
    grad = calculate_gradient(y, tx, w)
    b = np.linalg.norm(grad)
    while b > eps:
        counter += 1
        grad = calculate_gradient(y, tx, w)
        w -= gamma * grad
        b = np.linalg.norm(grad)
        print(f"Gradient norm = {b}\r", end="")
        if counter == max_iter:
            print("reached max_iter")
            break
    return calculate_loss(y, tx, w), w, np.linalg.norm(grad)


def stochastic_gradient_descent(y, tx, w, batch_size, max_iter, gamma, eps=1e-4):
    counter = 0
    grad = calculate_gradient(y, tx, w)
    b = np.linalg.norm(grad)
    while b > eps:
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad = calculate_gradient(y_batch, tx_batch, w)
            b = np.linalg.norm(grad)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            #print(f"Gradient norm = {b}\r", end="")
            counter += 1
        if counter == max_iter:
            print("reached max_iter")
            break
    return calculate_loss(y, tx, w), w, np.linalg.norm((1/len(y))*calculate_gradient(y, tx, w))


def is_cat(x, length=10):
    """Check if an array is categorical"""
    boolean_index = list([])
    if x.shape == (len(x),):
        if len(set(x)) < length:
            boolean = True
        else:
            boolean = False
            boolean_index.append(boolean)
    else:
        for i in range(x.shape[1]):
            if len(set(x[:, i])) < length:
                boolean = True
            else:
                boolean = False
            boolean_index.append(boolean)
    return np.array(boolean_index)


def build_poly(x, degree, pairwise_interaction=True, intercept=False):
    null, x_rNa = row_na_omit(x[:, 0], x)
    cat_index = is_cat(x_rNa)
    categorical_variables = x[:, cat_index]
    continuous_variables = x[:, ~cat_index]
    augmented_x = continuous_variables
    if degree > 1:
        for i in range(2, degree+1):
            augmented_x = np.c_[augmented_x, np.power(continuous_variables, i)]
    if pairwise_interaction:
        for j in tqdm(range(continuous_variables.shape[1])):
            for k in range(continuous_variables.shape[1]):
                if j >= k:
                    continue
                else:
                    augmented_x = np.c_[augmented_x, np.multiply(
                        continuous_variables[:, j], continuous_variables[:, k])]
    if intercept:
        inter = np.ones((x.shape[0], 1))
        augmented_x = np.c_[np.ones((x.shape[0], 1)), augmented_x]
    augmented_x = np.c_[augmented_x, categorical_variables]
    return augmented_x

def threshold(y, fitted_probabilities, step = 0.01):
    """find the best threshold for classification"""
    candidates = np.arange(0.2, 0.8, step)
    thresholds = list([])
    accuracies = list([])
    for i in tqdm(candidates):
        prediction = (fitted_probabilities>i)*1
        accuracy = 1 - sum(np.abs(prediction - y))/len(y)
        thresholds.append(i)
        accuracies.append(accuracy)
    index = accuracies.index(max(accuracies))
    return thresholds[index]

## Data processing

### Data importation

In [ ]:
y_train, x_train, ids_train = load_csv_data("data/train.csv")
x_train[np.where(x_train == -999)] = np.nan
y_train[np.where(y_train == -1)] = 0

### Creating a clean dataset without missing values

In [ ]:
y_train_rNA, x_train_rNA = row_na_omit(y_train, x_train)

### Column selection

In [ ]:
# This can be done on train and test.
x_train = col_na_omit(x_train, tol = 1)

### Data spliting

In [ ]:
x_train, x_te, y_train, y_te = split_data(x_train, y_train, 0.8, seed = 1)

### Outliers

In [ ]:
# Can't be done on test set.
y_train, x_train, x_train_rNA = outliers(y_train, x_train, x_train_rNA, quantile = 3)

# Gaussian scaling, imputation and skewness analysis

## Mean imputation

In [ ]:
# x_train_mean = mean_imputation(x_train, x_train_rNA)
# x_te_mean = mean_imputation(x_te, x_train_rNA)

## Median imputation

In [ ]:
# Originaly, median_imputation was scaling and imputing. But here I wanted to impute on the original scale.
# Hence, I define a knew median_imputation() function.
def median_imputation(na_data, clean_data):
    median = np.median(clean_data, axis=0)
    for i in range(na_data.shape[1]):
        na_data[:, i] = np.nan_to_num(na_data[:, i], nan=median[i])
    return(na_data)

# Imputation on the original scale
x_train_median = median_imputation(x_train, x_train_rNA)
x_te_median = median_imputation(x_te, x_train_rNA)

## Stochastic kmeans imputation

In [ ]:
# x_train = stochastic_kmeans_imputation(na_data = x_train, clean_data = x_train_rNA, length=1000)
# x_te = stochastic_kmeans_imputation(na_data = x_te, clean_data = x_train_rNA, length=1000)

# Data augmentation and scaling

In [ ]:
# Augmentation on the original scale
x_train_augmented = build_poly(x = x_train_median, degree = 5, pairwise_interaction = True)
x_te_augmented = build_poly(x = x_te_median, degree = 5, pairwise_interaction = True)
x_train_rNA_augmented = build_poly(x = x_train_rNA, degree = 5, pairwise_interaction = True)
 
# Scaling
null, mean_, std_ = gaussian_scaling(x_train_rNA_augmented)
x_train_augmented = (x_train_augmented - mean_) / std_
x_te_augmented = (x_te_augmented - mean_) / std_

In [ ]:
# Under normality assumption, Gaussian scaling makes sense.
# print(np.min(x_train_augmented, axis=0), "\n", "\n", np.max(x_train_augmented, axis=0))

In [ ]:
# skewness(x_train_augmented)

In [ ]:
# Density example, symmetry seems to be "good enough".
# pd.DataFrame(x_train_augmented.T[0]).plot(kind='density')

# Quick EDA

In [ ]:
# eda = sv.analyze(pd.concat([pd.DataFrame({"Prediction" : y_train}), pd.DataFrame(x_train)], axis=1))
# eda.show_html('eda.html')

# Depency structure

In [ ]:
# # We need SVD for fast convergence.
# df = pd.DataFrame(x_train_augmented)
# f = plt.figure(figsize=(10, 10))
# plt.matshow(df.corr(), fignum=f.number)
# plt.xticks(range(df.shape[1]), df.columns, fontsize=10, rotation=45)
# plt.yticks(range(df.shape[1]), df.columns, fontsize=10)
# cb = plt.colorbar()
# cb.ax.tick_params(labelsize=10)
# plt.title('Correlation Matrix', fontsize=10);

# Singular Value Decomposition

In [ ]:
def eigen(A):
    """Simple eigenvalue decomposition and sorting by importance"""
    eigenValues, eigenVectors = np.linalg.eig(A)
    idx = np.argsort(eigenValues)
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return (eigenValues, eigenVectors)

cov = np.cov(x_train_augmented, rowvar=False)
eigenvalues, cov_eigenvectors = eigen(cov)

In [ ]:
x_train_changed = x_train_augmented.dot(cov_eigenvectors.real[:,sum(eigenvalues<1e-2):len(eigenvalues)])
x_te_changed = x_te_augmented.dot(cov_eigenvectors.real[:,sum(eigenvalues<1e-2):len(eigenvalues)])

# Gradient descent

In [ ]:
loss, w, grad_norm = gradient_descent(y_train,
                                      tx = x_train_changed,
                                      w = np.zeros(x_train_changed.shape[1]),
                                      max_iter = 10,
                                      gamma = 0.05,
                                      eps = 1e-4)
# print(loss,"\n", "\n", w,"\n", "\n", grad_norm)

# Performance

In [ ]:
thresh = threshold(y_train, sigmoid(x_train_changed@w))
pred = sigmoid(x_train_changed@w)
pred = (pred>thresh)*1
print(f"In sample performance : {1 - sum(np.abs(pred - y_train))/len(y_train)}")

In [ ]:
pred = sigmoid(x_te_changed@w)
pred = (pred>thresh)*1
print(f"Out of sample performance : {1 - sum(np.abs(pred - y_te))/len(y_te)}")

# Recompute on full dataset

In [ ]:
y , x, ids = load_csv_data("data/train.csv")
x[np.where(x == -999)] = np.nan
y[np.where(y == -1)] = 0

y_rNA, x_rNA = row_na_omit(y, x)

# y, x, x_rNA = outliers(y, x, x_rNA, quantile = 3)

x_median = median_imputation(x, x_rNA)

x_augmented = build_poly(x = x_median, degree = 5, pairwise_interaction = True)
x_rNA_augmented = build_poly(x = x_rNA, degree = 5, pairwise_interaction = True)


null, mean_, std_ = gaussian_scaling(x_rNA_augmented)
x_augmented = (x_augmented - mean_) / std_

cov = np.cov(x_augmented, rowvar=False)
eigenvalues, cov_eigenvectors = eigen(cov)

x_augmented_changed = x_augmented.dot(cov_eigenvectors.real[:,sum(eigenvalues<1e-2):len(eigenvalues)])

loss, w, grad_norm = gradient_descent(y,
                                      tx = x_augmented_changed,
                                      w = np.zeros(x_augmented_changed.shape[1]),
                                      max_iter = 10,
                                      gamma = 0.05,
                                      eps = 1e-4)

thresh = threshold(y, sigmoid(x_augmented_changed@w))
pred = sigmoid(x_augmented_changed@w)
pred = (pred>thresh)*1
print(f"In sample performance : {1 - sum(np.abs(pred - y))/len(y)}")

# Test set

In [ ]:
#load test set
y_pred, x_test, ids_test = load_csv_data("data/test.csv")

In [ ]:
#make sure to apply the same imputaion changes to the test set
x_test[np.where(x_test == -999)] = np.nan
x_test = median_imputation(x_test, x_train_rNA)

In [ ]:
#applying singular value decomposition as well (not sure if correct)
x_test_augmented = build_poly(x = x_test, degree = 5, pairwise_interaction = True)
x_test_augmented = (x_test_augmented - mean_) / std_
x_test_changed = x_test_augmented.dot(cov_eigenvectors.real[:,sum(eigenvalues<1e-2):len(eigenvalues)])

In [ ]:
#create prediction
y_pred = sigmoid(x_test_changed@w)
y_pred = (y_pred>thresh)*1

print(y_pred)

In [ ]:
#convert vectors for submition
y_pred[np.where(y_pred == 0)] = -1

In [ ]:
#Observe the distribution of the classes
print(len(y_pred[np.where(y_pred == 1)])/len(y_pred[np.where(y_pred == -1)]))

In [ ]:
create_csv_submission(ids_test, y_pred, "anthony_submission_06_10_5.csv")